In [1]:
library(tidyverse)
library(repr)
library(readxl)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [36]:
hungry <- read.csv("data/processed.hungarian.data", header = FALSE)

colnames(hungry) <- c("age", 'sex', 'chest_pain_type', 'resting_bps', 'cholestoral', 'fasting_blood_sugar', 'rest_ecg', 'max_heartrate', 'excercise_angina', 'depression', 'slope', 'major_vessels', 'thal', 'heart_disease')


#age, cp, rest_bps, chol, max_hr, heart_disease

#hw1 coding team: visualize data, wrangle, summarize, split training and testing data
# replace '?' to na

hungry_clean <- hungry |>
    select(-sex, -excercise_angina, -depression, -slope, -major_vessels, -thal, -rest_ecg, -fasting_blood_sugar) #|>
   # replace_with_na(replace = list(cholestoral = "?")
hungry_clean[hungry_clean == "?"] <- NA
hungry_clean

age,chest_pain_type,resting_bps,cholestoral,max_heartrate,heart_disease
<int>,<int>,<chr>,<chr>,<chr>,<int>
28,2,130,132,185,0
29,2,120,243,160,0
29,2,140,NA,170,0
30,1,170,237,170,0
31,2,100,219,150,0
32,2,105,198,165,0
32,2,110,225,184,0
32,2,125,254,155,0
33,3,120,298,185,0
